In [ ]:
from pyspark.sql import SparkSession

In [ ]:
jars = [
    '/usr/share/aws/redshift/jdbc/RedshiftJDBC.jar',
    '/usr/share/aws/redshift/spark-redshift/lib/spark-redshift.jar',
    '/usr/share/aws/redshift/spark-redshift/lib/spark-avro.jar',
    '/usr/share/aws/redshift/spark-redshift/lib/minimal-json.jar'
]

In [ ]:
spark = SparkSession. \
    builder. \
    appName('Redshift Integration'). \
    master('yarn'). \
    config('spark.jars', ','.join(jars)). \
    getOrCreate()

In [ ]:
import boto3
import json
sm_client = boto3.client('secretsmanager')
secret_value = sm_client.get_secret_value(SecretId='demo/aigithub/redshift')
credentials = json.loads(secret_value['SecretString'])

username = credentials['username']
password = credentials['password']
host = credentials['host']
port = credentials['port']
database = 'github_dm'
url = f"jdbc:redshift://{host}:{port}/{database}?user={username}&password={password}"

In [ ]:

df = spark. \
    read. \
    format('io.github.spark_redshift_community.spark.redshift'). \
    option(
        'aws_iam_role', 
        'arn:aws:iam::269066542444:role/service-role/AmazonRedshift-CommandsAccessRole-20220625T110940'
    ). \
    option('url', url). \
    option('dbtable', 'public.ghrepos'). \
    option('tempdir', 's3://aigithub/temp/ghrepos'). \
    load()

In [ ]:
df.printSchema()

In [ ]:
df.show()